# Data Gathering

In [1]:
from fastai.tabular import *

In [2]:
df = pd.read_pickle('./data/ligamx/WLT2')

In [3]:
df_ns = pd.read_pickle('./data/ligamx/WLT_new_season')

In [4]:
df = pd.concat([df, df_ns])
df.reset_index(drop = True, inplace = True)

In [5]:
df.at[4948, "JRD"] = 4
df.at[4949, "JRD"] = 4

In [6]:
# sorting
import numpy as np
df['sort_val'] = df.ANO + df.JRD/1000 + (df.TIPO == 'apertura')/10
df = df.sort_values('sort_val').drop('sort_val', 1)

In [7]:
df.reset_index(drop=True, inplace=True)

In [8]:
teams = []

for name in df.LOC.unique():
    curr_df_loc = df[df.LOC == name].copy()
    curr_df_loc["ES_LOC"] = True
    curr_df_vis = df[df.VIS == name].copy()
    curr_df_vis["ES_LOC"] = False
    
    curr_df = pd.concat([curr_df_loc, curr_df_vis]).copy()
    curr_df.LOC = name
    del curr_df['VIS']
    curr_df['sort_val'] = curr_df.ANO + curr_df.JRD/1000 + (curr_df.TIPO == 'apertura')/10
    curr_df = curr_df.sort_values('sort_val').drop('sort_val', 1)
    curr_df.reset_index(drop = True, inplace = True)
    teams.append(curr_df)

# Data Prep

In [1]:
from fastai import *
from fastai.text import *
from fastai.tabular import *

In [2]:
deep_vars = ['RES']
#categorical values
cat_names = ['LOC', 'JRD', 'ANO', 'TIPO', 'ES_LOC']
#continious variables
cont_names = ['L_JJ', 'L_POS', 'L_R_JG_JJ', 'L_R_JE_JJ', 'L_R_JP_JJ', 'L_R_GF_JJ', 'L_R_GC_JJ', 'L_R_DIF_JJ', 'L_R_PTS_JJ',
              'V_JJ', 'V_POS', 'V_R_JG_JJ', 'V_R_JE_JJ', 'V_R_JP_JJ', 'V_R_GF_JJ', 'V_R_GC_JJ', 'V_R_DIF_JJ', 'V_R_PTS_JJ']
procs = [Categorify, Normalize]

# data = (TabularList.from_df(df, path="./data/ligamx", cat_names=cat_names, cont_names=cont_names, procs=procs)
#    .split_by_idx(range(len(df)-35, len(df)))
#    .label_from_df(cols=deep_vars)
#    .databunch(bs=64))

# learn = tabular_learner(data, layers=[300, 200, 100],  ps=[0.1, 0.1, 0.1], emb_drop=0.1, metrics=accuracy)

In [3]:
def orig_create(train_ds:Dataset, valid_ds:Dataset, test_ds:Optional[Dataset]=None, path:PathOrStr='.', bs:int=64,
           val_bs:int=None, num_workers:int=defaults.cpus, dl_tfms:Optional[Collection[Callable]]=None,
           device:torch.device=None, collate_fn:Callable=data_collate, no_check:bool=False, **dl_kwargs)->'DataBunch':
    "Create a `DataBunch` from `train_ds`, `valid_ds` and maybe `test_ds` with a batch size of `bs`. Passes `**dl_kwargs` to `DataLoader()`"
    datasets = DataBunch._init_ds(train_ds, valid_ds, test_ds)
    orig_create.ds = datasets
    val_bs = ifnone(val_bs, bs)
    dls = [DataLoader(d, b, shuffle=s, drop_last=s, num_workers=num_workers, **dl_kwargs) for d,b,s in zip(datasets, (bs,val_bs,val_bs,val_bs), (True,False,False,False)) if d is not None]
    return DataBunch(*dls, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [4]:
def my_create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', no_check:bool=False, bs=64, val_bs:int=None,
               num_workers:int=0, device:torch.device=None, collate_fn:Callable=data_collate,
               dl_tfms:Optional[Collection[Callable]]=None, bptt:int=70, backwards:bool=False, **dl_kwargs) -> DataBunch:
    "Create a `TextDataBunch` in `path` from the `datasets` for language modelling. Passes `**dl_kwargs` on to `DataLoader()`"
    datasets = cls._init_ds(train_ds, valid_ds, test_ds)
    val_bs = ifnone(val_bs, bs)
    datasets = [LanguageModelPreLoader(ds, shuffle=(i==0), bs=(bs if i==0 else val_bs), bptt=bptt, backwards=backwards)
                for i,ds in enumerate(datasets)]
    val_bs = bs
    dls = [DataLoader(d, b, shuffle=False, **dl_kwargs) for d,b in zip(datasets, (bs,val_bs,val_bs,val_bs)) if d is not None]
    return cls(*dls, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [5]:
LanguageModelPreLoader??

In [14]:
# DataBunch.create = orig_create

In [15]:
data = (TabularList.from_df(teams[0], path="./data/ligamx", cat_names=cat_names, cont_names=cont_names, procs=procs)
   .split_none()
   .label_from_df(cols=deep_vars))

In [16]:
data.databunch(bs=64)

TabularDataBunch;

Train: LabelList (459 items)
x: TabularList
LOC chiapas; JRD 1.0; ANO 2004; TIPO clausura; ES_LOC True; L_JJ -1.6398; L_POS -1.7630; L_R_JG_JJ -1.3698; L_R_JE_JJ -1.2386; L_R_JP_JJ -1.5161; L_R_GF_JJ -2.0064; L_R_GC_JJ -1.9935; L_R_DIF_JJ 0.2426; L_R_PTS_JJ -1.8302; V_JJ -1.6398; V_POS -1.6684; V_R_JG_JJ -1.3776; V_R_JE_JJ -1.2408; V_R_JP_JJ -1.4959; V_R_GF_JJ -1.9463; V_R_GC_JJ -2.1048; V_R_DIF_JJ 0.0909; V_R_PTS_JJ -1.8672; ,LOC chiapas; JRD 2.0; ANO 2004; TIPO clausura; ES_LOC False; L_JJ -1.4406; L_POS -0.8645; L_R_JG_JJ 3.1717; L_R_JE_JJ -1.2386; L_R_JP_JJ -1.5161; L_R_GF_JJ -0.2852; L_R_GC_JJ -1.9935; L_R_DIF_JJ 1.4507; L_R_PTS_JJ 2.8475; V_JJ -1.4406; V_POS -1.4888; V_R_JG_JJ 2.7670; V_R_JE_JJ -1.2408; V_R_JP_JJ -1.4959; V_R_GF_JJ 4.4351; V_R_GC_JJ 1.1541; V_R_DIF_JJ 2.6257; V_R_PTS_JJ 2.5631; ,LOC chiapas; JRD 3.0; ANO 2004; TIPO clausura; ES_LOC True; L_JJ -1.2413; L_POS -0.6848; L_R_JG_JJ 0.9009; L_R_JE_JJ -1.2386; L_R_JP_JJ 0.5290; L_R_GF_JJ 2.2967; L_R_GC

In [17]:
data.x._bunch.create.ds[0]

AttributeError: 'function' object has no attribute 'ds'

In [ ]:
LabelList??

In [ ]:
Dataset??

In [ ]:
ItemLists??

In [ ]:
data.x._bunch??

In [ ]:
data.databunch??

In [ ]:
teams[0].head(70)

In [ ]:
data = data.databunch(bs=64)

In [ ]:
DeviceDataLoader??

In [ ]:
it = iter(data.train_dl)

In [ ]:
curr = next(it)

In [ ]:
curr[0]

In [ ]:
teams[0]

# Non fastai data prep

In [18]:
from fastai import *

In [19]:
cdf = teams[0]

In [20]:
cdf.tail()

,LOC,JRD,ANO,TIPO,L_POS,L_JJ,L_R_JG_JJ,L_R_JE_JJ,L_R_JP_JJ,L_R_GF_JJ,...,V_JJ,V_R_JG_JJ,V_R_JE_JJ,V_R_JP_JJ,V_R_GF_JJ,V_R_GC_JJ,V_R_DIF_JJ,V_R_PTS_JJ,RES,ES_LOC
454,chiapas,13.0,2017,clausura,4.0,12.0,0.416667,0.416667,0.166667,1.583333,...,12.0,0.333333,0.166667,0.500000,0.916667,1.500000,-0.583333,1.166667,2,False
455,chiapas,14.0,2017,clausura,15.0,13.0,0.307692,0.153846,0.538462,0.923077,...,13.0,0.461538,0.307692,0.230769,1.923077,1.384615,0.538462,1.692308,0,True
456,chiapas,15.0,2017,clausura,13.0,14.0,0.285714,0.214286,0.500000,1.142857,...,14.0,0.285714,0.142857,0.571429,0.928571,1.714286,-0.785714,1.000000,1,False
457,chiapas,16.0,2017,clausura,17.0,15.0,0.266667,0.200000,0.533333,1.000000,...,15.0,0.333333,0.600000,0.066667,1.400000,1.066667,0.333333,1.600000,1,True
458,chiapas,17.0,2017,clausura,3.0,16.0,0.437500,0.312500,0.250000,1.500000,...,16.0,0.250000,0.250000,0.500000,1.062500,1.750000,-0.687500,1.000000,0,False


In [21]:
deep_vars = ['RES']
#categorical values
cat_names = ['LOC', 'JRD', 'ANO', 'TIPO', 'ES_LOC']
#continious variables
cont_names = ['L_JJ', 'L_POS', 'L_R_JG_JJ', 'L_R_JE_JJ', 'L_R_JP_JJ', 'L_R_GF_JJ', 'L_R_GC_JJ', 'L_R_DIF_JJ', 'L_R_PTS_JJ',
              'V_JJ', 'V_POS', 'V_R_JG_JJ', 'V_R_JE_JJ', 'V_R_JP_JJ', 'V_R_GF_JJ', 'V_R_GC_JJ', 'V_R_DIF_JJ', 'V_R_PTS_JJ']
procs = [Categorify, Normalize]

In [22]:
def my_create(train_ds:Dataset, valid_ds:Dataset, test_ds:Optional[Dataset]=None, path:PathOrStr='.', bs:int=64,
           val_bs:int=None, num_workers:int=defaults.cpus, dl_tfms:Optional[Collection[Callable]]=None,
           device:torch.device=None, collate_fn:Callable=data_collate, no_check:bool=False, **dl_kwargs)->'DataBunch':
    "Create a `DataBunch` from `train_ds`, `valid_ds` and maybe `test_ds` with a batch size of `bs`. Passes `**dl_kwargs` to `DataLoader()`"
    datasets = DataBunch._init_ds(train_ds, valid_ds, test_ds)
    val_bs = ifnone(val_bs, bs)
    dls = [DataLoader(d, b, shuffle=s, drop_last=s, num_workers=num_workers, **dl_kwargs) for d,b,s in zip(datasets, (bs,val_bs,val_bs,val_bs), (False,False,False,False)) if d is not None]
    return DataBunch(*dls, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [23]:
data.x._bunch.create??

In [51]:
data = (TabularList.from_df(cdf, path="./data/ligamx", cat_names=cat_names, cont_names=cont_names, procs=procs)
       .split_none()
       .label_from_df(cols=deep_vars))

In [25]:
data.x._bunch.create = my_create

In [52]:
data = data.databunch(bs=len(cdf))

In [27]:
curr_dta = next(iter(data.train_dl))

In [28]:
curr_dta[0][1][range(0, 10)]

tensor([[-1.6398, -1.7630, -1.3698, -1.2386, -1.5161, -2.0064, -1.9935,  0.2426,
         -1.8302, -1.6398, -1.6684, -1.3776, -1.2408, -1.4959, -1.9463, -2.1048,
          0.0909, -1.8672],
        [-1.4406, -0.8645,  3.1717, -1.2386, -1.5161, -0.2852, -1.9935,  1.4507,
          2.8475, -1.4406, -1.4888,  2.7670, -1.2408, -1.4959,  4.4351,  1.1541,
          2.6257,  2.5631],
        [-1.2413, -0.6848,  0.9009, -1.2386,  0.5290,  2.2967,  0.9243,  0.8466,
          0.5087, -1.2413, -1.1294,  2.7670, -1.2408, -1.4959, -0.3510, -2.1048,
          1.3583,  2.5631],
        [-1.0421,  0.2138,  0.1440,  0.2968, -0.1527,  0.8623,  0.4380,  0.2426,
          0.2488, -1.0421, -1.4888,  1.3855, -1.2408, -0.0360,  1.7762,  0.6110,
          0.9359,  1.0863],
        [-0.8429, -1.5833,  2.0363, -1.2386, -0.4935,  1.4361, -0.1699,  1.1486,
          1.6781, -0.8429,  1.7456, -1.3776,  1.0807,  0.6939, -0.7498,  0.3394,
         -0.8596, -1.1288],
        [-0.6437, -1.4036,  1.3551,  0.6039, -1.51

In [29]:
ds_teams = []

for cdf in teams:
    data = (TabularList.from_df(cdf, path="./data/ligamx", cat_names=cat_names, cont_names=cont_names, procs=procs)
       .split_none()
       .label_from_df(cols=deep_vars))
    data = data.databunch(bs=len(cdf))
    ds_teams.append(next(iter(data.train_dl)))

## Custom dataset and dataloader

In [30]:
import math
class RNNTabDataset(Dataset):
    def __init__(self, dfs, bptt):
        self.dfs = dfs
        self.bs = len(dfs)
        self.bptt = bptt
        
        self.largest= -1;
        for i in range(self.bs): self.largest=max(self.largest, len(dfs[i][0][0]))
        
        self.fix_bptt()
        self.process()
        
    def __getitem__(self, idx):
        #return ((self.data[0][0][idx], self.data[0][1][idx]), self.data[1][idx])
        return self.data[idx]
        
    def __len__(self):
        return len(self.data)
    
    def get_split(self, sub_df, batch, ttl_batches):
        ttl = len(sub_df)
        
        pos_btchs = math.ceil(ttl / float(self.bptt))
        incomplete_btchs = ttl_batches - pos_btchs
        partial_to_use =  ttl % self.bptt
        partial_to_fake = self.bptt - partial_to_use
        repl = sub_df[0] # may be changed by an average function later
        
        #pure repl
        if batch <= incomplete_btchs : return torch.stack([repl] * self.bptt)
        
        #partial repl
        elif batch == incomplete_btchs + 1 and partial_to_use != 0:
            return torch.cat( (torch.stack([repl] * partial_to_fake), sub_df[0:partial_to_use]), 0)
            
        #no repl
        start = (batch-1-incomplete_btchs) * self.bptt#(
        end = (batch-incomplete_btchs) * self.bptt #]
        
        if partial_to_use != 0:
            start = start - partial_to_fake
            end = end - partial_to_fake
        
        if(start >= ttl): print("xd")
        return sub_df[start : end]
    
    def process(self):
        self.data = []
        ttl_batches = math.ceil(self.largest / float(self.bptt))
        for batch in range(1, ttl_batches+1):
            for b in range(self.bs):
                cat = self.get_split(self.dfs[b][0][0], batch, ttl_batches)
                cont = self.get_split(self.dfs[b][0][1], batch, ttl_batches)
                y = self.get_split(self.dfs[b][1], batch, ttl_batches)
                if(len(y) != self.bptt): print(batch, b)
                self.data.append(((cat, cont), y))
    
    def prnt(self):
        print(self.data)
    
    def fix_bptt(self):
        #TODO
        self.ds_len = math.ceil(self.largest / float(self.bptt)) * self.bs * self.bptt

In [31]:
rnn_tab_ds = RNNTabDataset(ds_teams, 20)

In [32]:
for i in range(len(rnn_tab_ds.data)):
    print(i, rnn_tab_ds.data[i][1])

0 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
1 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
2 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
3 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
4 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
5 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
6 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
7 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
8 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
9 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
10 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
11 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
12 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
13 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0])
14

469 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
470 tensor([0, 1, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0, 2, 2, 2])
471 tensor([2, 1, 2, 2, 2, 0, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2, 0, 2, 1, 1])
472 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
473 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
474 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
475 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
476 tensor([2, 0, 1, 2, 0, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 0, 1, 2, 0, 2])
477 tensor([2, 1, 2, 0, 2, 0, 2, 0, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 2, 2])
478 tensor([2, 2, 0, 0, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 0, 0, 2, 2, 1])
479 tensor([2, 0, 2, 2, 2, 0, 2, 1, 2, 1, 0, 0, 2, 2, 1, 1, 0, 1, 2, 0])
480 tensor([0, 0, 0, 1, 1, 2, 1, 0, 1, 1, 0, 1, 2, 1, 2, 1, 1, 1, 2, 1])
481 tensor([0, 2, 2, 1, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 1, 2, 1, 1, 1, 2])
482 tensor([1, 2, 2, 0, 1, 2, 0, 1, 0, 1, 0, 2, 1, 

In [33]:
for i in range(28):
    if(len(ds_teams[27][1]) <= 20): print('a')

In [34]:
deep_vars = ['RES']
#categorical values
cat_names = ['LOC', 'JRD', 'ANO', 'TIPO', 'ES_LOC']
#continious variables
cont_names = ['L_JJ', 'L_POS', 'L_R_JG_JJ', 'L_R_JE_JJ', 'L_R_JP_JJ', 'L_R_GF_JJ', 'L_R_GC_JJ', 'L_R_DIF_JJ', 'L_R_PTS_JJ',
              'V_JJ', 'V_POS', 'V_R_JG_JJ', 'V_R_JE_JJ', 'V_R_JP_JJ', 'V_R_GF_JJ', 'V_R_GC_JJ', 'V_R_DIF_JJ', 'V_R_PTS_JJ']
procs = [Categorify, Normalize]

In [35]:
data = (TabularList.from_df(cdf, path="./data/ligamx", cat_names=cat_names, cont_names=cont_names, procs=procs)
       .split_none()
       .label_from_df(cols=deep_vars)
       .databunch(bs=28))

In [36]:
curr_dl = DataLoader(rnn_tab_ds, 28, shuffle=False, drop_last = False)

In [37]:
rnn_tab_ds[26][1][:20]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
data.train_dl = curr_dl

In [ ]:
class TabularModelX(Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs:ListSizes, n_cont:int, out_sz:int, layers:Collection[int], ps:Collection[float]=None,
                 emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False):
        super().__init__()
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
        self.embeds = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(emb_drop)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range
        sizes = self.get_sizes(layers, out_sz)
        actns = [nn.ReLU(inplace=True) for _ in range(len(sizes)-2)] + [None]
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        if bn_final: layers.append(nn.BatchNorm1d(sizes[-1]))
        self.layers = nn.Sequential(*layers)

    def get_sizes(self, layers, out_sz):
        return [self.n_emb + self.n_cont] + layers + [out_sz]

    def forward(self, x_cat:Tensor, x_cont:Tensor) -> Tensor:
        print(x_cat)
        print(x_cont)
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        x = self.layers(x)
        if self.y_range is not None:
            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x

In [ ]:
def my_tabular_learner(data:DataBunch, layers:Collection[int], emb_szs:Dict[str,int]=None, metrics=None,
        ps:Collection[float]=None, emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, **learn_kwargs):
    "Get a `Learner` using `data`, with `metrics`, including a `TabularModel` created using the remaining params."
    #emb_szs = data.get_emb_szs(ifnone(emb_szs, {}))
    model = TabularModelX(emb_szs, len(data.cont_names), out_sz=data.c, layers=layers, ps=ps, emb_drop=emb_drop,
                         y_range=y_range, use_bn=use_bn)
    return Learner(data, model, metrics=metrics, **learn_kwargs)

In [ ]:
learn = my_tabular_learner(data, layers=[500, 200, 100],  ps=[0.1, 0.1, 0.1], emb_drop=0.1, metrics=accuracy)

In [ ]:
data = data.databunch(bs=len(cdf))

# Model

## Creation

In [154]:
class RnnTabularModel(nn.Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs:ListSizes, n_cont:int, out_sz:int, lyrs:int, bs:int, ps:Collection[float]=None,
                 emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False):
        super().__init__()
        self.embeds = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(emb_drop)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range

        #rnn stuff
        self.nh = n_cont
        for ni, nf in emb_szs: self.nh += nf
        self.rnn = nn.GRU(self.nh, self.nh, lyrs, batch_first=True)
        self.h_o = nn.Linear(self.nh,out_sz)
        self.bn = BatchNorm1dFlat(self.nh)
        self.h = torch.zeros(lyrs, bs, self.nh)#.cuda()


    def forward(self, x_cat:Tensor, x_cont:Tensor) -> Tensor:
        i_proc = torch.zeros(x_cat.shape[0], x_cat.shape[1], self.nh);
        
        for i_bptt in range(x_cat.shape[1]):
            curr_cat = x_cat[:,i_bptt,:]
            curr_cont = x_cont[:,i_bptt,:]
            
            if self.n_emb != 0:
                x = [e(curr_cat[:,i]) for i,e in enumerate(self.embeds)]
                x = torch.cat(x, 1)
                x = self.emb_drop(x)
            if self.n_cont != 0:
                curr_cont = self.bn_cont(curr_cont)
                x = torch.cat([x, curr_cont], 1) if self.n_emb != 0 else curr_cont
            i_proc[:, i_bptt] = x
            
        res,h = self.rnn(i_proc, self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [64]:
nn.GRU()

In [ ]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

## Use of model

In [57]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [55]:
deep_vars = ['RES']
#categorical values
cat_names = ['LOC','JRD', 'ANO', 'TIPO']
#continious variables
cont_names = ['L_JJ', 'L_POS', 'L_R_JG_JJ', 'L_R_JE_JJ', 'L_R_JP_JJ', 'L_R_GF_JJ', 'L_R_GC_JJ', 'L_R_DIF_JJ', 'L_R_PTS_JJ',
              'V_JJ', 'V_POS', 'V_R_JG_JJ', 'V_R_JE_JJ', 'V_R_JP_JJ', 'V_R_GF_JJ', 'V_R_GC_JJ', 'V_R_DIF_JJ', 'V_R_PTS_JJ']

In [69]:
emb_szs = [(29, 15), (21, 9), (18, 8), (3, 3)]
n_cont =len(cont_names)
out_sz = 3 #total possibilities
lyrs = 2
bs = 28

In [281]:
# Instantiate the model with hyperparameters
model = RnnTabularModel(emb_szs, n_cont, out_sz, lyrs, bs)

# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
n_epochs = 100
lr=1e-3

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [282]:
# Training Run
model.train()
for epoch in range(1, n_epochs + 1):
    dl_it = iter(curr_dl)    
    
    for i_batch in range(len(dl_it)):    
        optimizer.zero_grad() # Clears existing gradients from previous epoch
        curr_batch = next(dl_it)
        x_cat, x_cont, res = curr_batch[0][0].to(device), curr_batch[0][1].to(device), curr_batch[1].to(device)
        output = model(x_cat, x_cont)
        
        loss = criterion(output.view(560,3), res.view(-1).long())
        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly
    
    if epoch%1 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 1/100............. Loss: 1.1546
Epoch: 2/100............. Loss: 1.1518
Epoch: 3/100............. Loss: 1.1747
Epoch: 4/100............. Loss: 1.2026
Epoch: 5/100............. Loss: 1.2358
Epoch: 6/100............. Loss: 1.2513
Epoch: 7/100............. Loss: 1.1656
Epoch: 8/100............. Loss: 1.1731
Epoch: 9/100............. Loss: 1.1681
Epoch: 10/100............. Loss: 1.1510
Epoch: 11/100............. Loss: 1.1263
Epoch: 12/100............. Loss: 1.1009
Epoch: 13/100............. Loss: 1.1078
Epoch: 14/100............. Loss: 1.1073
Epoch: 15/100............. Loss: 1.1270
Epoch: 16/100............. Loss: 1.1065
Epoch: 17/100............. Loss: 1.0617
Epoch: 18/100............. Loss: 1.0889
Epoch: 19/100............. Loss: 1.0603
Epoch: 20/100............. Loss: 1.0584
Epoch: 21/100............. Loss: 1.0550
Epoch: 22/100............. Loss: 1.0197
Epoch: 23/100............. Loss: 1.0467
Epoch: 24/100............. Loss: 1.0330
Epoch: 25/100............. Loss: 1.0205
Epoch: 26

In [289]:
# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def predict(model, data_cat, data_cont, data_res):
    model.eval()
    out = model(data_cat.to(device), data_cont.to(device))
    prob = nn.functional.softmax(out.view(560, 3), dim=0).data
    # Taking the class with the highest probability score from the output
    out = torch.max(prob, dim=1)[1]
    return out

In [181]:
tst

[[tensor([[[1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           ...,
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2]],
  
          [[1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           ...,
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2]],
  
          [[1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           ...,
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2],
           [1, 1, 1, 2, 2]],
  
          ...,
  
          [[1, 1, 1, 1, 2],
           [1, 1, 1, 1, 2],
           [1, 1, 1, 1, 2],
           ...,
           [1, 1, 1, 1, 2],
           [1, 1, 1, 1, 2],
           [1, 1, 1, 1, 2]],
  
          [[1, 1, 1, 1, 2],
           [1, 1, 1, 1, 2],
           [1, 1, 1, 1, 2],
           ...,
           [1, 1, 1, 1, 2],
           [1, 1, 1, 1, 2],
           [1, 1, 1, 1, 2]],
  
          [[1, 1, 1, 1, 1],
           [1,

In [288]:
curr_it = iter(curr_dl)
for i in range(len(curr_it)):
    tst = next(curr_it)

In [208]:
sample_cat = tst[0][0][15][19]
sample_cont = tst[0][1][15][19]
sample_res = tst[1][15][19]

In [225]:
len(sample_cont)

18

In [221]:
sample_cont[None,None]

tensor([[[-0.8220,  0.5123, -0.4454, -0.1394,  0.8653,  1.0890,  0.9801,
           0.1642, -0.5185, -0.8218,  0.7856, -0.5247, -0.1176,  0.9761,
          -0.8542,  0.2569, -0.9460, -0.6017]]])

In [290]:
sample_pred = predict(model, (tst[0][0][15][19])[None, None], (tst[0][1][15][19])[None, None], tst[1][15][19])

RuntimeError: Expected hidden size (2, 1, 53), got (2, 28, 53)

In [266]:
tst[1].view(-1)

tensor([0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 1, 1, 0, 2, 0, 1, 1, 0, 2, 2, 1, 2,
        0, 2, 2, 0, 0, 1, 2, 1, 2, 2, 1, 0, 1, 2, 0, 1, 2, 2, 2, 2, 0, 2, 1, 0,
        2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 1, 1, 1, 1, 0, 1, 2, 0, 2, 1, 0, 2, 0, 2,
        0, 1, 2, 0, 1, 2, 2, 0, 0, 0, 2, 1, 2, 2, 0, 1, 2, 1, 0, 0, 2, 0, 2, 0,
        0, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 0, 0, 0, 0, 2, 1, 0, 0, 1,
        0, 2, 2, 1, 2, 0, 1, 2, 0, 0, 1, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 1, 1,
        0, 2, 2, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 0, 1, 2, 1,
        2, 2, 2, 0, 1, 0, 2, 2, 1, 1, 2, 1, 1, 1, 1, 0, 2, 2, 2, 1, 2, 2, 2, 0,
        0, 2, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1, 2, 2, 0,
        0, 0, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 0, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1,
        2, 0, 1, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 2, 0, 1, 1, 2, 1, 1, 1, 1, 1,
        1, 0, 0, 2, 0, 1, 2, 1, 1, 0, 0, 1, 2, 2, 2, 1, 0, 2, 0, 2, 0, 2, 0, 0,
        2, 1, 2, 2, 1, 1, 0, 2, 2, 1, 0,

In [261]:
sample_res

tensor([0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 1, 1, 0, 2, 0, 2, 0, 0, 1, 1, 1, 1,
        1, 0, 2, 0, 0, 1, 2, 1, 2, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 2, 2, 1, 2, 0, 1, 1, 1, 0, 0, 1, 2, 0, 1, 1, 0, 0, 0, 1,
        0, 1, 2, 0, 0, 2, 2, 0, 0, 2, 2, 1, 0, 2, 1, 1, 1, 1, 0, 0, 2, 1, 2, 0,
        0, 2, 2, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1,
        2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 1, 0,
        0, 2, 2, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 0, 1, 2, 0, 0, 2, 1,
        2, 2, 2, 0, 0, 0, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0,
        0, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 0, 2, 0, 0, 1, 0, 2, 2, 1, 2, 0, 0,
        0, 0, 2, 1, 0, 2, 2, 2, 2, 1, 1, 2, 1, 0, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1,
        1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 2, 1, 2, 0, 1, 0,
        0, 2, 0, 2, 0, 1, 2, 1, 0, 0, 0, 1, 2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 0, 0,
        2, 1, 2, 2, 1, 1, 0, 2, 0, 0, 0,

In [286]:
ttl = 0
for i in range(len(sample_res)):
    if(tst[1].view(-1)[i] == sample_res[i]): ttl += 1;
print(ttl)
print(ttl/560)

401
0.7160714285714286


In [284]:
x_cat, x_cont, opt_res = tst[0][0].to(device), tst[0][1].to(device), tst[1].to(device)
res = model(x_cat, x_cont)

In [174]:
a = res.view(560,3)

In [175]:
a.shape

torch.Size([560, 3])

In [176]:
a

tensor([[ 0.4455, -0.0132,  0.0848],
        [ 0.4438, -0.0148,  0.0774],
        [ 0.4437, -0.0166,  0.0717],
        ...,
        [-2.3829, -0.3268, -0.1882],
        [-2.3756, -0.3514, -0.2349],
        [-2.2101, -0.2192, -0.1468]], grad_fn=<ViewBackward>)

In [177]:
res

tensor([[[ 4.4550e-01, -1.3243e-02,  8.4849e-02],
         [ 4.4380e-01, -1.4818e-02,  7.7392e-02],
         [ 4.4372e-01, -1.6578e-02,  7.1684e-02],
         ...,
         [ 4.4873e-01, -1.8039e-02,  7.1475e-02],
         [ 4.5020e-01, -3.0685e-02,  7.4440e-02],
         [ 4.4876e-01, -1.5992e-02,  8.8905e-02]],

        [[ 6.4799e-01,  1.5648e-01,  3.1094e-02],
         [ 6.4524e-01,  1.8014e-01,  3.2276e-02],
         [ 6.4783e-01,  1.9683e-01,  3.3386e-02],
         ...,
         [ 6.7868e-01,  2.1301e-01,  3.0935e-02],
         [ 6.8152e-01,  1.9582e-01,  5.0746e-02],
         [ 6.5946e-01,  1.4105e-01,  3.3128e-02]],

        [[ 1.3796e-02,  3.2704e-01, -1.4078e-01],
         [ 1.8715e-03,  3.4906e-01, -1.2398e-01],
         [-2.0986e-02,  3.7055e-01, -1.2127e-01],
         ...,
         [-7.5900e-02,  4.0827e-01, -1.7442e-01],
         [-7.2977e-02,  3.8313e-01, -1.4696e-01],
         [-7.5574e-03,  3.2474e-01, -1.6616e-01]],

        ...,

        [[ 3.3565e-01, -1.8572e-01,  3

In [178]:
loss_func = nn.CrossEntropyLoss()
loss_func(res.view(560,3), opt_res.view(-1))

tensor(1.1363, grad_fn=<NllLossBackward>)

In [153]:
nn.CrossEntropyLoss??

In [147]:
len(opt_res.view(-1))

560

In [148]:
len(res.view(-1))

560

In [83]:
tst[0][0].shape

torch.Size([28, 20, 5])

In [84]:
tst[0][1].shape

torch.Size([28, 20, 18])

In [127]:
tst[1].view(-1)

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,